# Aggregating and putting together all format data

Going to put it all together now I think

In [68]:
import pandas as pd 
import json 
import requests 

In [69]:
#api call and cleaning
bulk = requests.get('https://api.scryfall.com/bulk-data') #I learned that the uri for bulk data changes daily, so requesting the bulk data for the day then getting the uri for updated cards is needed (I had outdated data)
bulk_df = pd.json_normalize(bulk.json(),'data')
oracle_uri = bulk_df['download_uri'].loc[bulk_df.query('type == "oracle_cards"').index[0]]

response = requests.get(oracle_uri)
raw_oracle_cards = pd.json_normalize(response.json())

oracle_cards = raw_oracle_cards[[#'object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id',
       #'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 
       'name', 
       #'lang',
       #'released_at', 'uri', 'scryfall_uri', 
       'layout', 
       #'highres_image',
       #'image_status', 'mana_cost', 'cmc', 
       'type_line', 
       #'oracle_text',
       #'colors', 'color_identity', 'keywords', 'games', 'reserved',
       #'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint',
       #'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri',
       #'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       #'prints_search_uri', 'collector_number', 'digital', 
       #'rarity',
       #'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       #'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       #'booster', 'story_spotlight', 'edhrec_rank', 'image_uris.small',
       #'image_uris.normal', 'image_uris.large', 'image_uris.png',
       #'image_uris.art_crop', 'image_uris.border_crop',
       'legalities.standard',
       #'legalities.future', 'legalities.historic',
       #'legalities.gladiator', 
       'legalities.pioneer',
       #'legalities.explorer', 
       'legalities.modern', 
       'legalities.legacy',
       'legalities.pauper', 
       'legalities.vintage', 
       #'legalities.penny',
       #'legalities.commander', 'legalities.brawl',
       #'legalities.historicbrawl', 'legalities.alchemy',
       #'legalities.paupercommander', 'legalities.duel',
       #'legalities.oldschool', 'legalities.premodern', 
       'prices.usd',
       #'prices.usd_foil', 'prices.usd_etched', 'prices.eur',
       #'prices.eur_foil', 
       'prices.tix', 
       #'related_uris.gatherer',
       #'related_uris.tcgplayer_infinite_articles',
       #'related_uris.tcgplayer_infinite_decks', 'related_uris.edhrec',
       #'security_stamp', 'preview.source', 'preview.source_uri',
       #'preview.previewed_at', 'power', 'toughness', 'penny_rank',
       #'arena_id', 'watermark', 'produced_mana', 'all_parts',
       'card_faces', 
       #'frame_effects', 'tcgplayer_etched_id',
       #'promo_types', 'loyalty', 'life_modifier', 'hand_modifier',
       #'attraction_lights', 'color_indicator', 'content_warning'
       ]]
oracle_cards = oracle_cards.rename(columns = {
    'name':'Name',
    'rarity':'Rarity',
    'legalities.pauper':'Pauper_Legal',
    'legalities.standard':'Standard_Legal',
    'legalities.pioneer':'Pioneer_Legal',
    'legalities.modern':'Modern_Legal',
    'legalities.legacy':'Legacy_Legal',
    'legalities.vintage':'Vintage_Legal',
    'prices.usd':'Price_USD',
    'prices.tix':'Price_Tix'
})

oracle_cards = oracle_cards.query("Standard_Legal == 'legal' or Pioneer_Legal == 'legal' or Modern_Legal == 'legal' or Legacy_Legal == 'legal' or Vintage_Legal == 'restricted' or Vintage_Legal == 'legal'")

card_faces = pd.json_normalize(oracle_cards['card_faces'].loc[~oracle_cards['card_faces'].isna()])
front_face = pd.json_normalize(card_faces[0])
back_face = pd.json_normalize(card_faces[1])

faces_index = 0 # we lose the relationship between the indices when we normalize the faces data, I'm not sure how to fix this with pandas but just keeping track of it like this works fine
for card_index in oracle_cards.loc[~oracle_cards['card_faces'].isna()].index:
    if oracle_cards['layout'].loc[card_index] == 'split':
        oracle_cards['Name'].loc[card_index] = front_face['name'].loc[faces_index]+'/'+back_face['name'].loc[faces_index]
    else:
        oracle_cards["Name"].loc[card_index] = front_face['name'].loc[faces_index]
    faces_index += 1


oracle_cards.head()

,Name,layout,type_line,Standard_Legal,Pioneer_Legal,Modern_Legal,Legacy_Legal,Pauper_Legal,Vintage_Legal,Price_USD,Price_Tix,card_faces
0,Static Orb,normal,Artifact,not_legal,not_legal,not_legal,legal,not_legal,legal,21.46,0.51,NaN
1,Sensory Deprivation,normal,Enchantment — Aura,not_legal,legal,legal,legal,legal,legal,0.02,0.04,NaN
2,Road of Return,normal,Sorcery,not_legal,not_legal,not_legal,legal,not_legal,legal,0.25,0.66,NaN
3,Storm Crow,normal,Creature — Bird,not_legal,not_legal,legal,legal,legal,legal,0.21,0.03,NaN
4,Walking Sponge,normal,Creature — Sponge,not_legal,not_legal,not_legal,legal,not_legal,legal,0.21,0.09,NaN


In [70]:
#adding Is_Land values
oracle_cards['Is_Land'] = ''

for i in oracle_cards.index:
    if 'Land' in oracle_cards['type_line'].loc[i]:
        oracle_cards['Is_Land'].loc[i] = True
    else:
        oracle_cards['Is_Land'].loc[i] = False

oracle_cards = oracle_cards.drop(['type_line','layout','card_faces'], axis = 1)

In [71]:
format_names = ['Pauper','Standard','Pioneer','Modern','Legacy','Vintage']
pauper_deck_names = ['Burn','Affinity','Dimir Control','Mono-Blue Faeries','Orzhov Ephemerate','Gruul Ponza','Familiars',
         'Boros Synthesizer','Mono-Red Synthesizer','Walls Combo','Tron','Caw Gates','4 Color Ephemerate','Ephemerate Tron',
         'Bogles','Kiln Fiend','Boros Bully','Izzet Faeries','Goblins','Slivers','Jeskai Ephemerate']
standard_deck_names = ['Grixis Midrange','Mono-White Midrange','5 Color Midrange','Mono-Red Aggro',
        'Esper Midrange','Mono-Blue Tempo','Azorius Soldiers','Selesnya Poison',
        'Jund Midrange','Kami War','Esper Legends','Mardu Midrange','Rakdos Aggro',
        'Domain Control','Esper Aggro']
pioneer_deck_names = ['Rakdos Midrange','Abzan Greasefang','Nykthos Ramp','Azorius Control','Temur Indomitable Creativity',
         'Lotus Field Combo','Gruul Midrange','Izzet Phoenix','5 Color Midrange','Selesnya Angels',
         'Mono-White Humans','Dimir Control','Azorius Spirits','5 Color Fires of Invention','Mono-Red Goblins',
         'Mono-Blue Spirits','Rakdos Sacrifice','4 Color Auras','Mono-Black Midrange','Bant Spirits']
modern_deck_names = ['Izzet Murktide','5 Color Indomitable Creativity','Hammer Time','Rakdos Midrange','Burn','4 Color Omnath',
         'Amulet Titan','Golgari Yawgmoth','Domain Zoo','Mono-Green Tron','Dimir Mill','Generic Ragavan','Living End',
         'Merfolk',"Death's Shadow",'Jeskai Control','5 Color Reanimator','Mono-Blue Affinity','Jeskai Prowess','Azorius Control']
legacy_deck_names = ['Reanimator','Cephalid Breakfast','Painter','Mono-Blue 8-Cast','Izzet Delver','Sneak and Show','Death\'s Shadow',
         'Five-Color Zenith','Doomsday','Death and Taxes','Temur Delver','Mono-Red Prison','Elves','Naya Depths',
         'Lands','Food Chain Goblins','Maverick','Jeskai Control','4 Color Aluren','Mono-Red Burn','Mono-White Hammer',
         'Jeskai Stoneblade','4 Color Stoneblade','Omni-Tell','Ad Nauseam Tendrils']
vintage_deck_names = ['Esper Tinker','Mono-White Initiative','Paradoxical Outcome','Doomsday','Squee Hollow Vine','Dredge','Jeskai Control',
         'Oath of Druids','Prison Shops','Aggro Shops','4 Color Tempo','Azorius Initiative','Hogaak Vine']

formats = pd.DataFrame({'format_name': format_names,
                       'format_decklists' : [pauper_deck_names,standard_deck_names,pioneer_deck_names,modern_deck_names,legacy_deck_names,vintage_deck_names]})

formats.head()


,format_name,format_decklists
0,Pauper,"[Burn, Affinity, Dimir Control, Mono-Blue Faer..."
1,Standard,"[Grixis Midrange, Mono-White Midrange, 5 Color..."
2,Pioneer,"[Rakdos Midrange, Abzan Greasefang, Nykthos Ra..."
3,Modern,"[Izzet Murktide, 5 Color Indomitable Creativit..."
4,Legacy,"[Reanimator, Cephalid Breakfast, Painter, Mono..."


In [72]:
decks = pd.DataFrame(columns=['Deck_Name','Deck_List','Format','Paper_Price','MTGO_Price_Tix'])

deck_names_column = []
deck_lists_column = []
format_column = []

for format in formats.index:
    for deck in formats['format_decklists'].loc[format]:
        deck_names_column.append(deck)
        deck_lists_column.append(pd.read_csv("Deck Lists/"+formats['format_name'].loc[format]+" Metagame Decks/"+deck+".csv"))
        format_column.append(formats['format_name'].loc[format])

decks['Deck_Name'] = deck_names_column
decks['Deck_List'] = deck_lists_column
decks['Format'] = format_column

decks.head(len(decks))

,Deck_Name,Deck_List,Format,Paper_Price,MTGO_Price_Tix
0,Burn,Quantity Card_Name 0 ...,Pauper,NaN,NaN
1,Affinity,Quantity Card_Name 0 3...,Pauper,NaN,NaN
2,Dimir Control,Quantity Card_Name 0 ...,Pauper,NaN,NaN
3,Mono-Blue Faeries,Quantity Card_Name 0 ...,Pauper,NaN,NaN
4,Orzhov Ephemerate,Quantity Card_Name 0 2...,Pauper,NaN,NaN
...,...,...,...,...,...
109,Prison Shops,Quantity Card_Name ...,Vintage,NaN,NaN
110,Aggro Shops,Quantity Card_Name 0 ...,Vintage,NaN,NaN
111,4 Color Tempo,Quantity Card_Name 0 ...,Vintage,NaN,NaN
112,Azorius Initiative,Quantity Card_Name 0 ...,Vintage,NaN,NaN


In [76]:
def paper_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in decklist.index: #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = oracle_cards.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        if oracle_cards['Price_USD'].loc[price_index]:
            card_price = float(oracle_cards['Price_USD'].loc[price_index])
        else:
            card_price = 0

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

def mtgo_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in range(len(decklist.index)): #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = oracle_cards.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        if oracle_cards['Price_Tix'].loc[price_index]:
            card_price = float(oracle_cards['Price_Tix'].loc[price_index])
        else:
            card_price = 0

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

In [74]:
for i in decks.index:
    decks['Paper_Price'].loc[i] = paper_price(decks['Deck_List'].loc[i])
    decks['MTGO_Price_Tix'].loc[i] = mtgo_price(decks['Deck_List'].loc[i])

Card has no price: Great Furnace
Card has no price: Great Furnace
Card has no price: Scoured Barrens
Card has no price: Dust to Dust
Card has no price: Thermokarst
Card has no price: Tranquil Cove
Card has no price: Dust to Dust
Card has no price: Great Furnace
Card has no price: Tinder Wall
Card has no price: Dust to Dust
Card has no price: Dust to Dust
Card has no price: Dismal Backwater
Card has no price: Wind-Scarred Crag
Card has no price: Dust to Dust
Card has no price: Great Furnace
Card has no price: Blossoming Sands
Card has no price: Horned Sliver
Card has no price: Muscle Sliver
Card has no price: Winged Sliver
Card has no price: Clot Sliver
Card has no price: Dust to Dust
Card has no price: Winged Sliver
Card has no price: Elspeth, Sun's Champion
Card has no price: Memnite
Card has no price: Badlands
Card has no price: Lotus Petal
Card has no price: Scrubland
Card has no price: Underground Sea
Card has no price: Nomads en-Kor
Card has no price: Tundra
Card has no price: Und

In [75]:
decks[['Deck_Name','Format','Paper_Price','MTGO_Price_Tix']].to_csv("Format Price Data/Format_Price_Comparison.csv", index = False)